In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import (
    precision_recall_fscore_support,
    roc_auc_score,
    confusion_matrix,
    roc_curve,
    classification_report,
    f1_score
)
from transformers import RobertaTokenizer, RobertaModel
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    torch.cuda.empty_cache()


In [ ]:
df = pd.read_csv('vulnerability_dataset_all.csv')
print(f"Dataset Size: {df.shape}")
print(df['label'].value_counts(normalize=True))


df = df.dropna(subset=['function_code', 'label'])
df['function_code'] = df['function_code'].astype(str)
df['label'] = df['label'].astype(int)


In [ ]:
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = RobertaModel.from_pretrained("microsoft/codebert-base").to(device)
codebert_model.eval()

def get_embeddings(texts, batch_size=8, max_length=512):
    all_embeddings = []
    with torch.no_grad():
        for i in tqdm(range(0, len(texts), batch_size), desc="Getting Embedding"):
            batch_texts = texts[i:i+batch_size]
            inputs = tokenizer(
                batch_texts, 
                padding=True, 
                truncation=True, 
                max_length=max_length, 
                return_tensors="pt"
            ).to(device)
            outputs = codebert_model(**inputs)
            embeddings = outputs.last_hidden_state[:, 0, :].cpu()
            all_embeddings.append(embeddings)
    return torch.cat(all_embeddings, dim=0)


In [ ]:
sample_size = None
texts = df['function_code'].tolist()[:sample_size] if sample_size else df['function_code'].tolist()
labels = df['label'].tolist()[:sample_size] if sample_size else df['label'].tolist()


print("Starting embedding process...")
embeddings = get_embeddings(texts)
labels = torch.tensor(labels, dtype=torch.float32)

del codebert_model
torch.cuda.empty_cache()


In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(
    embeddings, labels, test_size=0.3, random_state=42, stratify=labels
)
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print(f"Train: {X_train.shape}, Validation: {X_val.shape}, Test: {X_test.shape}")


In [ ]:
class CodeVulnerabilityDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features.float()
        self.labels = labels.float().unsqueeze(1)
    def __len__(self):
        return len(self.labels)
    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

train_dataset = CodeVulnerabilityDataset(X_train, y_train)
val_dataset = CodeVulnerabilityDataset(X_val, y_val)
test_dataset = CodeVulnerabilityDataset(X_test, y_test)

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [ ]:
class EnhancedClassifier(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.main = nn.Sequential(
            nn.Linear(input_dim, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.3),
            
            nn.Linear(256, 256),
            nn.BatchNorm1d(256),
            nn.GELU(),
            nn.Dropout(0.3),
            
            nn.Linear(256, 128),
            nn.BatchNorm1d(128),
            nn.GELU(),
            nn.Dropout(0.2),
            
            nn.Linear(128, 1)
        )
    def forward(self, x):
        return self.main(x)


model = EnhancedClassifier(input_dim=embeddings.shape[1]).to(device)
print(model)


In [ ]:
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, alpha=0.25, logits=True, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        self.logits = logits
        self.reduction = reduction

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduction='none')
        
        pt = torch.exp(-BCE_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss


In [ ]:
criterion = FocalLoss(gamma=2.0, alpha=0.25, logits=True).to(device)

optimizer = optim.AdamW(model.parameters(), lr=2e-5, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=3, factor=0.5)


In [ ]:
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=30, patience=5):
    best_val_loss = float('inf')
    early_stop_counter = 0
    history = {'train_loss': [], 'val_loss': [], 'val_f1': []}

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0
        for x, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
            x, y = x.to(device), y.to(device)
            optimizer.zero_grad()
            outputs = model(x)
            loss = criterion(outputs, y)
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * x.size(0)
        
        train_loss /= len(train_loader.dataset)
        history['train_loss'].append(train_loss)
        
        val_loss, val_f1 = evaluate_loss(model, val_loader, criterion)
        history['val_loss'].append(val_loss)
        history['val_f1'].append(val_f1)
        scheduler.step(val_loss)
        
        print(f"Epoch {epoch+1}: Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val F1: {val_f1:.4f}")
        
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            early_stop_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            early_stop_counter += 1
            if early_stop_counter >= patience:
                print("Early stopping triggered!")
                break
    
    model.load_state_dict(torch.load('best_model.pth'))
    return history

def evaluate_loss(model, loader, criterion, threshold=0.5):
    model.eval()
    total_loss = 0.0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)
            outputs = model(x)
            loss = criterion(outputs, y)
            total_loss += loss.item() * x.size(0)

            preds = (torch.sigmoid(outputs) > threshold).float()
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(y.cpu().numpy())

    avg_loss = total_loss / len(loader.dataset)
    f1 = f1_score(all_labels, all_preds, zero_division=0)
    return avg_loss, f1


In [ ]:
history = train_model(model, train_loader, val_loader, criterion, optimizer)


In [ ]:
plt.figure(figsize=(10, 5))
plt.plot(history['train_loss'], label='Train Loss')
plt.plot(history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()


In [ ]:
def full_evaluation(model, loader, threshold=0.5):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            outputs = torch.sigmoid(model(x)).cpu().numpy()
            all_probs.extend(outputs)
            all_labels.extend(y.numpy())
    
    y_true = np.array(all_labels).flatten()
    y_pred = (np.array(all_probs).flatten() > threshold)
    

    print("\nClassification Report:")
    print(classification_report(y_true, y_pred))  


    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(6, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=['Non-Vulnerable', 'Vulnerable'],
                yticklabels=['Non-Vulnerable', 'Vulnerable'])
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()

    fpr, tpr, _ = roc_curve(y_true, all_probs)
    roc_auc = roc_auc_score(y_true, all_probs)
    plt.figure(figsize=(6, 6))
    plt.plot(fpr, tpr, label=f'ROC AUC = {roc_auc:.2f}')
    plt.plot([0, 1], [0, 1], 'k--')
    plt.title('ROC Curve')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return roc_auc


In [ ]:
print("\nValidation Set Evaluation:")
val_auc = full_evaluation(model, val_loader)

print("\nTest Set Evaluation:")
test_auc = full_evaluation(model, test_loader)


In [ ]:
def find_optimal_threshold(model, loader):
    model.eval()
    all_probs, all_labels = [], []
    with torch.no_grad():
        for x, y in loader:
            x = x.to(device)
            outputs = torch.sigmoid(model(x)).cpu().numpy()
            all_probs.extend(outputs)
            all_labels.extend(y.numpy())
    
    y_true = np.array(all_labels).flatten()
    probs = np.array(all_probs).flatten()
    
    thresholds = np.linspace(0.1, 0.9, 50)
    best_f1 = 0
    best_th = 0.5
    for th in thresholds:
        f1 = f1_score(y_true, probs > th, zero_division=0)
        if f1 > best_f1:
            best_f1 = f1
            best_th = th
    return best_th

optimal_th = find_optimal_threshold(model, val_loader)
print(f"\nOptimal Threshold: {optimal_th:.4f}")

print("\nTest Set (with Optimal Threshold):")
full_evaluation(model, test_loader, threshold=optimal_th)


In [ ]:
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'input_dim': embeddings.shape[1],
    'threshold': optimal_th
}, 'vulnerability_detector.pth')


In [ ]:
model.eval()
all_probs = []

with torch.no_grad():
    for x, y in val_loader:
        x = x.to(device)
        outputs = torch.sigmoid(model(x)).cpu().numpy()
        all_probs.extend(outputs.flatten())

y_pred_probs = np.array(all_probs)
y_true = y_val.numpy() if isinstance(y_val, torch.Tensor) else np.array(y_val)
precision, recall, thresholds = precision_recall_curve(y_true, y_pred_probs)

f1_scores = 2 * (precision * recall) / (precision + recall + 1e-8)


plt.plot(thresholds, f1_scores[:-1])
plt.xlabel("Threshold")
plt.ylabel("F1 Score")
plt.title("F1 Score vs Threshold (Validation Set)")
plt.grid(True)
plt.show()
